In [1]:
TARGET_LOCATION = 'Plymouth, MA'
TARGET_LOCATION = 'Jamaica Plain, MA'
TARGET_LOCATION = 'Andover, MA'
TARGET_LOCATION = 'Lynn, MA'
TARGET_LOCATION = 'Chelmsford, MA'
TARGET_LOCATION = 'Boulder, CO'
TARGET_LOCATION = 'Lafayette, CO'

from model_lib import RedfinModel

In [2]:
redfin = RedfinModel(TARGET_LOCATION, {
  'style': ['SINGLE_FAMILY', 'TOWNHOUSE'],
  'beds': [4, 5]
})
train_df = redfin.fetch_data('sold')
#
train_df.head(3)

Using cached data for Lafayette, CO sold as of 2024-08-05
filtering column: style allowed_values: ['SINGLE_FAMILY', 'TOWNHOUSE']
filtering column: beds allowed_values: [4, 5]
Filtered data shape: (120, 43) (from (363, 43))


,property_url,mls,mls_id,status,text,style,full_street_line,street,unit,city,...,parking_garage,agent,agent_email,agent_phones,broker,broker_phone,broker_website,nearby_schools,primary_photo,alt_photos
1,https://www.realtor.com/realestateandhomes-det...,DVCO,7076145,SOLD,Discover the perfect blend of charm and modern...,SINGLE_FAMILY,529 London Ave,529 London Ave,<NA>,Lafayette,...,2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Boulder Valley Re 2 School District,http://ap.rdcpix.com/f83d2384bfa509e4cabe2de45...,http://ap.rdcpix.com/f83d2384bfa509e4cabe2de45...
9,https://www.realtor.com/realestateandhomes-det...,DVCO,8832188,SOLD,"BRAND NEW HOME!! Introducing the Camellia, a v...",SINGLE_FAMILY,1300 Brookfield Pl,1300 Brookfield Pl,<NA>,Erie,...,2,TEAM KAMINSKY/ DOELL,team@landmarkcolorado.com,"[{'number': '7202487653', 'type': 'Mobile', 'e...",<NA>,<NA>,<NA>,Boulder Valley Re 2 School District,http://ap.rdcpix.com/377f6f3015b60a6613fd1636c...,http://ap.rdcpix.com/377f6f3015b60a6613fd1636c...
12,https://www.realtor.com/realestateandhomes-det...,DVCO,8675638,SOLD,!!READY SUMMER 2024!! This Avalon is waiting t...,SINGLE_FAMILY,2548 Sawyer Ln,2548 Sawyer Ln,<NA>,Erie,...,2,Matthew Tonge,matthew.tonge@mdch.com,"[{'number': '7199648296', 'type': None, 'ext':...",<NA>,<NA>,<NA>,Boulder Valley Re 2 School District,http://ap.rdcpix.com/454594de51747a3d6597f3e8f...,http://ap.rdcpix.com/454594de51747a3d6597f3e8f...


In [3]:
train_df.columns.values
# one hot encode columns

array(['property_url', 'mls', 'mls_id', 'status', 'text', 'style',
       'full_street_line', 'street', 'unit', 'city', 'state', 'zip_code',
       'beds', 'full_baths', 'half_baths', 'sqft', 'year_built',
       'days_on_mls', 'list_price', 'list_date', 'sold_price',
       'last_sold_date', 'assessed_value', 'estimated_value', 'lot_sqft',
       'price_per_sqft', 'latitude', 'longitude', 'neighborhoods',
       'county', 'fips_code', 'stories', 'hoa_fee', 'parking_garage',
       'agent', 'agent_email', 'agent_phones', 'broker', 'broker_phone',
       'broker_website', 'nearby_schools', 'primary_photo', 'alt_photos'],
      dtype=object)

In [4]:
redfin.train_from_raw(train_df, train_df[RedfinModel.TARGET_COLUMN])
print(redfin.trained_columns)


['assessed_value' 'beds' 'broker' 'broker_phone' 'broker_website'
 'days_on_mls' 'fips_code' 'full_baths' 'half_baths' 'hoa_fee' 'lot_sqft'
 'parking_garage' 'sqft' 'stories' 'text' 'unit' 'year_built' 'city=Erie'
 'city=Lafayette' 'state=CO' 'style=SINGLE_FAMILY']


In [5]:
mean_score = redfin.show_cross_validation(train_df.drop(RedfinModel.TARGET_COLUMN, axis=1), train_df[RedfinModel.TARGET_COLUMN])


Cross-validation scores: [-0.48945934  0.24942148  0.44955406  0.59879431 -0.48944332]
Average cross-validation score: 0.06


In [6]:
test_df = redfin.fetch_data('for_sale')
# test_df = redfin.filter_data(test_df)
result_df = redfin.predict(test_df)

Using cached data for Lafayette, CO for_sale as of 2024-08-05
filtering column: style allowed_values: ['SINGLE_FAMILY', 'TOWNHOUSE']
filtering column: beds allowed_values: [4, 5]
Filtered data shape: (56, 43) (from (148, 43))
Dropping columns: ['city=Louisville']
Adding columns: ['broker' 'broker_phone' 'broker_website']
Predicted 56 values


In [7]:
print(redfin.trained_columns)
# remove exponent formatting
result_df[RedfinModel.OUTPUT_COLUMNS].sort_values(by=['diff_percent'], ascending=False).head(25)


['assessed_value' 'beds' 'broker' 'broker_phone' 'broker_website'
 'days_on_mls' 'fips_code' 'full_baths' 'half_baths' 'hoa_fee' 'lot_sqft'
 'parking_garage' 'sqft' 'stories' 'text' 'unit' 'year_built' 'city=Erie'
 'city=Lafayette' 'state=CO' 'style=SINGLE_FAMILY']


,readable_address,style,sqft,beds,full_baths,list_price,predicted,diff,diff_percent,property_url
51,"1211 Nathan Pl, Erie, CO",SINGLE_FAMILY,3420,4,3,822950,1.084709e+06,261759.428889,31.807452,https://www.realtor.com/realestateandhomes-det...
74,"1740 Lyonesse St, Lafayette, CO",SINGLE_FAMILY,1824,4,2,540000,6.882366e+05,148236.559524,27.451215,https://www.realtor.com/realestateandhomes-det...
108,"1760 Shallot Cir, Lafayette, CO",SINGLE_FAMILY,1664,4,2,599900,7.339605e+05,134060.5,22.347141,https://www.realtor.com/realestateandhomes-det...
55,"1150 Colony Dr, Erie, CO",SINGLE_FAMILY,3380,5,3,812950,9.899842e+05,177034.174286,21.77676,https://www.realtor.com/realestateandhomes-det...
38,"1054 Iliad Way, Lafayette, CO",SINGLE_FAMILY,2189,4,2,669000,7.937985e+05,124798.54,18.65449,https://www.realtor.com/realestateandhomes-det...
77,"1337 Brookfield Pl, Lafayette, CO",SINGLE_FAMILY,2912,4,4,819410,9.647300e+05,145320.0,17.734712,https://www.realtor.com/realestateandhomes-det...
75,"1151 Nathan Pl, Erie, CO",SINGLE_FAMILY,3380,5,3,842950,9.899842e+05,147034.174286,17.442811,https://www.realtor.com/realestateandhomes-det...
5,"1301 Brookfield Pl, Lafayette, CO",SINGLE_FAMILY,2912,4,4,822240,9.647300e+05,142490.0,17.32949,https://www.realtor.com/realestateandhomes-det...
16,"1161 Nathan Pl, Erie, CO",SINGLE_FAMILY,3650,5,3,864950,9.979790e+05,133029.0,15.379964,https://www.realtor.com/realestateandhomes-det...
70,"1276 Brookfield Pl, Lafayette, CO",SINGLE_FAMILY,2912,4,4,839990,9.647300e+05,124740.0,14.850177,https://www.realtor.com/realestateandhomes-det...


In [8]:
importances = redfin.print_feature_importances()
print(f"===\nFeature Importances ({len(importances)})\n===")
for importance in importances:
    print(importance)



===
Feature Importances (3)
===
('broker', np.float64(0.8304599106059297))
('beds', np.float64(0.11243660564171402))
('assessed_value', np.float64(0.05710348375235628))


In [9]:
# test_df.head()
print(set(result_df['style']))

{'SINGLE_FAMILY'}


In [10]:
result_df[RedfinModel.OUTPUT_COLUMNS].to_dict(orient='records')[0]

{'readable_address': '1301 Brookfield Pl, Lafayette, CO',
 'style': 'SINGLE_FAMILY',
 'sqft': 2912,
 'beds': 4,
 'full_baths': 4,
 'list_price': 822240,
 'predicted': 964730.0,
 'diff': 142490.0,
 'diff_percent': 17.329490173185444,
 'property_url': 'https://www.realtor.com/realestateandhomes-detail/9005831795'}